# COBRE Schizophrenia Data Visualization

Welcome!
This notebook contains some interactive visualizations of the data from the COBRE dataset available on Nilearn. Run the whole notebook to start the visualizations and use the drop-down menus to toggle between Schizophrenia Patients and Controls.
**Note**: The call to the data below will load only 30 subjects in interest of time. If you would like to load more, change the `n_subject` parameter. If you would like to load all 146, set this parameter to 'None'

In [1]:
#import data
from nilearn import datasets
data = datasets.fetch_cobre(n_subjects=30)

#import atlas
parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')
atlas_filename = parcellations.scale064

/home/aalbury/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py:2372: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [2]:
import pandas as pd
# import cleaned phenotypic data csv file from repo
pheno = pd.read_csv('pheno.csv')

#create lists of filepaths for patients and controls
patients = []
controls = []

for i in pheno.index:
    if pheno.loc[i, 'subject_type']=='Patient':
    
        patients.append(pheno.loc[i, 'path'])
    else:
        controls.append(pheno.loc[i, 'path'])

The cell below generates a plotly express app to show the age distribution of subjects in the data. Use the dropdown box to select between Patients and Controls.

In [3]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = pheno
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Age"),
    dcc.Graph(id='graph'),
    html.Label([
        "Participant type",
        dcc.Dropdown(
            id='subject_type', clearable=False,
            value='Patient', options=[
                {'label': c, 'value': c}
                for c in df.subject_type.unique() #get all unique values from column
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("subject_type", "value")]
)
def update_figure(subject_type):
    return px.histogram(
        df[df["subject_type"]==subject_type], x="current_age", color="gender"
        
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [4]:
# function to plot average correlation matrix by subject type (Patient vs. Control)
def corr_matrix(subset):

    from nilearn.input_data import NiftiLabelsMasker
    from nilearn.connectome import ConnectivityMeasure

    func = []
    conf = []

    for i in range(len(data.func)):
        if data.func[i] in subset:
            func.append(data.func[i])

            conf.append(data.confounds[i])

    # ConenctivityMeasure from Nilearn uses simple 'correlation' to compute
    # connectivity matrices for all subjects in a list
    connectome_measure = ConnectivityMeasure(kind='correlation')

    # useful for plotting connectivity interactions on glass brain
    from nilearn import plotting

    # create masker to extract functional data within atlas parcels
    masker = NiftiLabelsMasker(labels_img=atlas_filename, 
                               standardize=True, 
                               memory='nilearn_cache', 
                               verbose=0)
    # extract time series from all subjects and concatenate them
    time_series = []
    for func, confounds in zip(func, conf):
        time_series.append(masker.fit_transform(func, confounds=confounds))
    # calculate correlation matrices across subjects and display
    correlation_matrices = connectome_measure.fit_transform(time_series)

    # Mean correlation matrix across 10 subjects can be grabbed like this,
    # using connectome measure object
    mean_correlation_matrix = connectome_measure.mean_

    # grab center coordinates for atlas labels
    coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_filename)

    import numpy as np
    # Mask the main diagonal
    np.fill_diagonal(mean_correlation_matrix, 0)

    plotting.plot_matrix(mean_correlation_matrix, figure=(10, 8),
                         vmax=0.8, vmin=-0.8, reorder=False, tri='lower')

Use the dropdown menu below to view an average correlation matrix for schizophrenia patients and healthy controls.

In [5]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

interact(corr_matrix, subset=[('Patients', patients), ('Controls', controls)]);

interactive(children=(Dropdown(description='subset', options=(('Patients', ['/home/aalbury/nilearn_data/cobre/…